In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [2]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
 pd.options.display.max_rows = 9999

In [4]:
from static_grader import grader

ModuleNotFoundError: No module named 'static_grader'

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We will be working with the same medical data used in the `pw` miniproject, but will be leveraging the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [5]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [6]:
import pandas as pd
import numpy as np
import gzip

In [7]:
# load the 2017 data
with gzip.open('./dw-data/201701scripts_sample.csv.gz', 'rb') as f:
    scripts = pd.read_csv(f)
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.390,0.470,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.380,4.070,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.500,1.400,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.620,2.440,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.160,0.260,6


In [8]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
with gzip.open('./dw-data/practices.csv.gz') as f:
    practices = pd.read_csv(f, names = col_names)
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [9]:
with gzip.open('./dw-data/chem.csv.gz') as f:
    chem = pd.read_csv(f)
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [10]:
summary_stats = [('items', (0,) * 6), ('quantity', (0,) * 6), ('nic', (0,) * 6), ('act_cost', (0,) * 6)]

In [11]:
summary_stats = scripts.describe().transpose()

In [12]:
summary_stats.reset_index(inplace = True)

In [13]:
totals = pd.DataFrame(scripts.agg({'items':'sum','quantity':'sum','nic':'sum','act_cost':'sum'}))
totals.reset_index(inplace = True)

In [14]:
summary_stats = summary_stats.merge(totals, how = 'left')
summary_stats.columns = ['metric','count','mean','std','min','25th','median','75th','max','total']

In [15]:
summary_stats = summary_stats[['metric','total','mean','std','25th','median','75th']]

In [16]:
summary_stats

,metric,total,mean,std,25th,median,75th
0,items,8888304.000,9.133,29.204,1.000,2.000,6.000
1,nic,71100424.840,73.059,188.070,7.800,22.640,65.000
2,act_cost,66164096.120,67.987,174.402,7.330,21.220,60.670
3,quantity,721457006.000,741.330,3665.427,28.000,100.000,350.000


In [17]:
summary_stats =  [('items', (8888304.000, 9.133,29.204 , 1.000, 2.000, 6.000)),
                  ('quantity', (721457006.000, 741.330, 3665.427, 28.000, 100.000, 350.000)),
                  ('nic', (71100424.840, 73.059, 188.070, 7.800, 22.640, 65.000)),
                  ('act_cost', (66164096.120, 67.987, 174.402, 7.330, 21.220, 60.670))]

In [18]:
grader.score.dw__summary_statistics(summary_stats)

NameError: name 'grader' is not defined

## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [19]:
result = pd.DataFrame(scripts.groupby('bnf_name')['items'].agg('sum'))
result = result.reset_index()
result = result.sort_values('items',ascending=False)

In [20]:
result.head()

,bnf_name,items
9033,Omeprazole_Cap E/C 20mg,218583
9460,Paracet_Tab 500mg,151669
924,Aspirin Disper_Tab 75mg,148591
11298,Simvastatin_Tab 40mg,132941
652,Amlodipine_Tab 5mg,128245


In [21]:
most_common_item = [("Omeprazole_Cap E/C 20mg", 218583)]

In [22]:
grader.score.dw__most_common_item(most_common_item)

NameError: name 'grader' is not defined

## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [25]:
practices_df =  practices[['code','post_code']]
practices_df = practices_df.drop_duplicates()
practices_df['rank'] = practices_df.groupby('code')['post_code'].rank()
practices_df = practices_df.query("rank == '1.0'")

TypeError: 'NoneType' object is not callable

In [26]:
joined_df = pd.merge(scripts,practices_df,how='left',left_on = "practice", right_on = "code")
result = pd.DataFrame(joined_df.groupby(['post_code','bnf_name'])['items'].agg('sum'))
result.reset_index(inplace = True)

In [27]:
total_item_post_code = pd.DataFrame(joined_df.groupby(['post_code'])['items'].agg('sum')).reset_index()

In [28]:
result['rank'] = result.groupby(['post_code'])['items'].rank(ascending = False)
result = result.query('rank == 1.0')
result = pd.merge(result, total_item_post_code, how= 'left', on="post_code")
result['share'] = result['items_x']/result['items_y']
result = result [['post_code','bnf_name', 'share']]
result = result.sort_values('post_code')
result = result.query('share > 0 ')
result = result.head(100)
postal_totals = [tuple(x) for x in result.values]

In [29]:
##items_by_region = [("B11 4BW", "Salbutamol_Inha 100mcg (200 D) CFF", 0.0310589063)] * 100
items_by_region = postal_totals

In [30]:
grader.score.dw__items_by_region(items_by_region)

NameError: name 'grader' is not defined

## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [31]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

In [32]:
opioids_join = '|'.join(opioids)
opioids_join

'morphine|oxycodone|methadone|fentanyl|pethidine|buprenorphine|propoxyphene|codeine'

In [33]:
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


In [34]:
opioid_codes = chem.loc[chem['NAME'].str.contains(opioids_join, case=False)]['CHEM SUB'].tolist()

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [35]:
practices_df =  practices[['code','name']]
practices_df = practices_df.drop_duplicates()
practices_df['rank'] = practices_df.groupby('code')['name'].rank()
practices_df = practices_df.query("rank == '1.0'")
practices_df = practices_df[['code','name']]
practices_df.columns = ['practice','practice_name']

TypeError: 'NoneType' object is not callable

In [36]:
scripts['flag'] = scripts['bnf_code'].isin(opioid_codes)

In [37]:
scripts.flag.sum()

34843

Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [38]:
opioids_per_practice = pd.DataFrame(scripts.groupby(['practice'])['flag'].agg(['mean','count']))
opioids_per_practice.reset_index(inplace=True)

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [39]:
overall_rate = scripts_chem_joined['flag'].mean()
opioids_per_practice['relative_rate'] = opioids_per_practice['mean'] - overall_rate

NameError: name 'scripts_chem_joined' is not defined

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [40]:
import statistics
import numpy as np

In [41]:
### std_deviation = opioids_per_practice['mean'].std()
std_deviation = scripts_chem_joined['flag'].std()
opioids_per_practice['std_error'] = std_deviation / np.sqrt(opioids_per_practice['count'])

NameError: name 'scripts_chem_joined' is not defined

In [42]:
opioids_per_practice['z_score'] = opioids_per_practice['relative_rate'] / opioids_per_practice['std_error']

KeyError: 'relative_rate'

In [43]:
opioids_per_practice = opioids_per_practice.sort_values('z_score', ascending= False)

KeyError: 'z_score'

In [ ]:
opioids_per_practice = opioids_per_practice.head(100)

In [ ]:
opioids_per_practice = pd.merge(opioids_per_practice,practices_df,how='left',left_on = "practice", right_on = "practice")

In [ ]:
opioids_per_practice = opioids_per_practice[['practice_name','z_score','count']]

In [ ]:
opioids_per_practice.head()

The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [ ]:
anomalies = [tuple(x) for x in opioids_per_practice.values]

In [ ]:
unique_practices = ...
anomalies = [("NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100

In [ ]:
grader.score.dw__script_anomalies(anomalies)

## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`.  Normalize the percent growth in prescriptions of individual items by the percent change in total number of prescriptions (think about whether this normalization should be a division or a subtraction). We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [18]:
# load the 2017 data
with gzip.open('./dw-data/201701scripts_sample.csv.gz', 'rb') as f:
    scripts_2017 = pd.read_csv(f)
scripts_2017.shape

(973193, 7)

In [19]:
# load the 2017 data
with gzip.open('./dw-data/201606scripts_sample.csv.gz', 'rb') as f:
    scripts_2016 = pd.read_csv(f)
scripts_2016.shape

(973193, 7)

In [27]:
items_2016 = pd.DataFrame(scripts_2016.groupby(['bnf_name'])['items'].agg('count')).reset_index()
total_2016 = items_2016['items'].sum()
items_2016['rate'] = items_2016['items']/total_2016

In [28]:
items_2017 = pd.DataFrame(scripts_2017.groupby(['bnf_name'])['items'].agg('count')).reset_index()
total_2017 = items_2017['items'].sum()
items_2017['rate'] = items_2017['items']/total_2017

In [29]:
items = items_2016.merge(items_2017, how = 'inner', on  = 'bnf_name')
items.columns = ['bnf_name','count_2016','rate_2016','count_2017','rate_2017']

In [30]:
items['growth'] = (items['rate_2017'] - items['rate_2016']) / items['rate_2016']

In [31]:
total_growth = (total_2017 - total_2016) / total_2016

In [34]:
items['relative_growth'] = items['growth'] - total_growth

In [35]:
items = items.query('count_2016 > 50')

In [36]:
items = items[['bnf_name', 'relative_growth', 'count_2016']]
items = items.sort_values('relative_growth', ascending = False)

In [37]:
high_growth_items = items.head(50)
high_shtinkage_items = items.tail(50)
script_growth = high_growth_items.append(high_shtinkage_items)

In [38]:
script_growth = [tuple(x) for x in script_growth.values] 

In [ ]:
##script_growth = [("Butec_Transdermal Patch 5mcg\/hr", 3.4677419355, 62.0)] * 100

In [40]:
grader.score.dw__script_growth(script_growth)

Your score:  1.0


In [42]:
del scripts_2016, scripts_2017, script_growth

## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [44]:
# load the 2017 data
with gzip.open('./dw-data/201701scripts_sample.csv.gz', 'rb') as f:
    scripts = pd.read_csv(f)
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.390,0.470,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.380,4.070,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.500,1.400,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.620,2.440,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.160,0.260,6


In [51]:
total_code = scripts.bnf_code.nunique()
p = 1/total_code
rates = scripts.groupby('bnf_code')['bnf_code'].count().rename('count_prescription').reset_index()
total_count = scripts['bnf_code'].count().astype(float)
rates['prescription_rate'] = rates.count_prescription / total_count

In [59]:
rare_codes = rates[rates.prescription_rate < 0.1*p]['bnf_code'].tolist()

In [60]:
#Creating 'rare' column in scripts
scripts['rare'] = scripts['bnf_code'].isin(rare_codes)

In [67]:
# Calculate total sum of act_cost for all treatment  and rate prescription
treatment_sum = scripts.groupby('practice')['act_cost'].sum()
rare_treatment_sum = scripts[scripts['rare']].groupby('practice')['act_cost'].sum()

In [73]:
rare_cost_prop = (rare_treatment_sum / treatment_sum)
rare_cost_prop.fillna(0, inplace=True)

mean_val = scripts[scripts['rare']]['act_cost'].sum() / scripts['act_cost'].sum()

relative_rare_cost_prop = rare_cost_prop - mean_val 
standard_errors = relative_rare_cost_prop.std()

mean_val, standard_errors

Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [ ]:
#rare_cost_prop = ...

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [ ]:
#relative_rare_cost_prop = ...

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [ ]:
#standard_errors = ...

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [ ]:
#rare_scores = ..#

In [75]:
rare_scores = (relative_rare_cost_prop / standard_errors).rename('z-score').reset_index()
rare_scores.columns =['code','z-score']
rare_scores.sort_values('z-score', ascending=False, inplace=True)

In [82]:
rare_scores = rare_scores.head(100)

In [88]:
rare_scores = pd.merge(rare_scores,practices_df,how='left',left_on = "code", right_on = "practice")

In [92]:
rare_scores = rare_scores[['code','practice_name','z-score']]

In [94]:
#rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100
rare_scripts = [tuple(x) for x in rare_scores.values]

In [95]:
grader.score.dw__rare_scripts(rare_scripts)

Your score:  1.0


*Copyright &copy; 2017 The Data Incubator.  All rights reserved.*